In [ ]:
############################## IMPORT ###################################

using Random
using Distributions


############################## STRUCTS ##################################

# Event Struct to store arrivals and departures
struct Event
    time::Float64  # Time of event (arrival or departure)
    is_arrival::Bool  # True if it's an arrival, False for a departure
    server_id::Int
    event_id::Int
    customer_id::Int
end

mutable struct Server
    id::Int 
    in_service::Int # Number of customers it keeps in service
    in_queue::Int # Number of customers it keeps in queue
    max_capacity::Int # Number of customers it can service at once
    max_queue_length::Int # Number of customers it can hold in the queue
end

# Define a mutable struct for QueueState with state logging and tracking different types of departures
mutable struct QueueState
    servers::Vector{Server}  # List of servers
    last_event_time::Float64  # Last event time
    total_queue_time::Float64 # Total time in the queue
    total_time::Float64  # Total simulation time / Total time in the system
    lost_customers::Int  # Number of customers who left due to full queues
    state_times::Dict{Int, Float64}  # Time spent in each state
    avg_queue_lengths::Vector{Float64} # A list of average queue lengths
end


########################### STATE INITIALIZING #############################

# Initialize servers
function init_servers(num_servers::Int, max_server_capacity::Int, max_queue_length::Int)
    return [Server(
        i, 
        0, 
        0, 
        max_server_capacity, 
        max_queue_length
        ) for i in 1:num_servers]
end

# Initialize queue state
function init_queue_state(num_servers::Int, max_server_capacity::Int, max_queue_length::Int)
    servers = init_servers(num_servers, max_server_capacity, max_queue_length)
    return QueueState(
        servers, 
        0.0, 
        0.0, 
        0.0, 
        0, 
        Dict{Int, Float64}(), 
        zeros(Float64, num_servers))
end


################################ LOGGING ######################################

# Log the state before processing an event
function log_event_before(event::Event, current_time::Float64, queue_state::QueueState)
    println("<br />########### Event: $(event.event_id) ###########")
    println("Customer ID: $(event.customer_id)")
    println("Current time: $current_time")
    if event.is_arrival
        println("Customer $(event.customer_id) is arriving at server $(event.server_id).")
    else
        println("Customer $(event.customer_id) is leaving from server $(event.server_id).")
    end
    println("System state before event:")
    for server in queue_state.servers
        println("  Server $(server.id): In Service = $(server.in_service), Queue Length = $(server.in_queue)")
    end
    println("Total lost customers so far: $(queue_state.lost_customers)")
end


# Log the state after processing an event
function log_event_after(current_time::Float64, queue_state::QueueState, average_queue_length::Float64)
    println("System state after event:")
    println("Current time: $current_time")
    for server in queue_state.servers
        println("  Server $(server.id): In Service = $(server.in_service), Queue Length = $(server.in_queue)")
    end
    println("Total lost customers: $(queue_state.lost_customers)")
    println("Total waiting time in queue: $(queue_state.total_queue_time)")
    println("Total simulation time: $(queue_state.total_time)")
    println("Average queue length: $average_queue_length")

    # Sort and print state times
    sorted_state_times = sort(collect(queue_state.state_times), by = x -> x[1])
    println("Sorted state times (State -> Time Spent):")
    for (state, time_spent) in sorted_state_times
        println("  State $state: $time_spent")
    end
    println("")
end


###################### UPDATE STATE TIMES ########################

function update_state_times!(queue_state::QueueState, time_in_queue::Float64)
    # Update state times
    current_state = sum(server.in_queue for server in queue_state.servers)
    if haskey(queue_state.state_times, current_state)
        queue_state.state_times[current_state] += time_in_queue
    else
        queue_state.state_times[current_state] = time_in_queue
    end
end


######################### CALCULATE STEADY STATE ##########################

function calculate_steady_state(avg_queue_lengths::Vector{Float64}, current_avg_queue_length::Float64, current_time::Float64; threshold::Float64=0.1)
    if length(avg_queue_lengths) < 5
        return (Float64[], nothing)
    end

    # looking at the recent 5 queue lengths
    recent_avg_lengths = avg_queue_lengths[end-4:end]
    deviations = [abs(current_avg_queue_length - avg) / avg for avg in recent_avg_lengths]

    # Finding the steady state by comparing the last 5 deviations with the treshold
    steady_state_time = if all(dev -> dev < threshold, deviations)
        current_time
    else
        nothing
    end

    return (deviations, steady_state_time)
end

function log_deviations(deviations::Vector{Float64})
    println("<br />Deviations:")
    for deviation in enumerate(deviations)
        #if i <= length(event_ids)
        println("Deviation: $deviation")
    end
end


################ EVENT HANDLING #################################

# Handle an arrival event
function handle_arrival!(queue_state::QueueState, server::Server)
    if server.in_service < server.max_capacity
        server.in_service += 1
    elseif server.in_queue < server.max_queue_length
        server.in_queue += 1
    else
        # Check if any other server has space in the queue
        placed_in_queue = false
        for other_server in queue_state.servers
            if other_server.id != server.id && other_server.in_queue < other_server.max_queue_length
                other_server.in_queue += 1
                placed_in_queue = true
                break
            end
        end
        # If no space was found in any queue, increment lost customers
        # The lost customer is never assigned to a server
        if !placed_in_queue
            queue_state.lost_customers += 1
        end
    end
end


# Handle a departure event
function handle_departure!(server::Server)
    if server.in_queue > 0 #If there are someone in the queue
        server.in_queue -= 1 # subtracts one from the queue
    else #If there are no one in the queue
        server.in_service -= 1 #We subtract one from the service
    end
end


############################### EVENT GENERATING ################################

# Generate arrival and departure events
function generate_events(arrival_times, service_times, num_servers)
    events = Vector{Event}()
    event_id_counter = 1
    last_departure_times = zeros(Float64, num_servers)
    max_departure_time = 0.0

    for i in 1:length(arrival_times)
        arrival_time = arrival_times[i]
        service_time = service_times[i]

        # Assign the server with the earliest predicted availability
        server_id = argmin(last_departure_times)
        
        # Customer ID is equal to the iteration through the for the loop, each iteration is one customer 
        customer_id = i

        # Creates an arrival event by setting isarrival to true
        push!(events, Event(arrival_time, true, server_id, event_id_counter, customer_id))
        event_id_counter += 1

        departure_time = max(arrival_time, last_departure_times[server_id]) + service_time
        
        # Update the list of departure times 
        last_departure_times[server_id] = departure_time

        # Update the max departure time
        max_departure_time = max(max_departure_time, departure_time)

        # Creates an departure event by setting isarrival to false
        push!(events, Event(departure_time, false, server_id, event_id_counter, customer_id))
        event_id_counter += 1
    end

    sort!(events, by = e -> e.time)
    #sorted_events = [Event(event.time, event.is_arrival, event.server_id, index, event.customer_id) for (index, event) in enumerate(events)]
    
    return events, max_departure_time
end


############################## EVENT PROCESSING ###############################

# Function to process an event in a queuing system
function process_event!(event, queue_state, time_limit, verbose)
    # Extract the current time from the event
    current_time = event.time

    # Check if the current time exceeds the time limit
    if current_time > time_limit
        println("Time limit reached: $time_limit")
        return false  # Stop processing further events
    end

    # Get the server associated with the event
    server = queue_state.servers[event.server_id]

    # Calculate the time elapsed since the last event
    time_in_queue = current_time - queue_state.last_event_time

    # Update the total time spent in queues for all servers
    queue_state.total_queue_time += sum(server.in_queue for server in queue_state.servers) * time_in_queue

    # Update the state of servers with the elapsed time
    update_state_times!(queue_state, time_in_queue)

    # Optionally log the event details before processing
    if verbose
        log_event_before(event, current_time, queue_state)
    end

    # Handle the event based on whether it's an arrival or departure
    if event.is_arrival
        handle_arrival!(queue_state, server)  # Process an arrival event
    else
        handle_departure!(server)  # Process a departure event
    end

    # Update the last event time to the current event's time
    queue_state.last_event_time = current_time

    # Update the total system time
    queue_state.total_time = current_time

    # Calculate the current average queue length
    current_avg_queue_length = queue_state.total_queue_time / queue_state.total_time

    # Store the current average queue length in the queue state
    push!(queue_state.avg_queue_lengths, current_avg_queue_length)

    # Check for steady-state conditions and calculate deviations
    deviations, steady_state_time = calculate_steady_state(queue_state.avg_queue_lengths, current_avg_queue_length, current_time)

    # Optionally log any deviations identified
    if verbose
        log_deviations(deviations)
    end

    # If steady-state is reached, log the time and stop processing
    if steady_state_time !== nothing
        println("Steady state reached at time: $steady_state_time")
        return false  # Indicate no further processing is needed
    end

    # Optionally log the event details after processing
    if verbose
        log_event_after(current_time, queue_state, current_avg_queue_length)
    end

    return true  # Indicate successful processing of the event
end

process_event! (generic function with 1 method)

In [2]:
###################### GENERATOR FOR DISTRIBUTIONS #########################

# Function to generate arrival times and service times
function generate_distributions(
    num_customers::Int,
    arrival_rate::Float64,
    service_mean::Float64,
)
    # Calculate the mean inter-arrival time
    interarrival_mean = 1.0 / arrival_rate

    # Generate inter-arrival times using an exponential distribution
    interarrival_times = rand(Exponential(interarrival_mean), num_customers)

    # Convert inter-arrival times to cumulative arrival times
    #arrival_times = interarrival_times
    arrival_times = cumsum(interarrival_times)

    # Generate random service times using an exponential distribution
    service_times = rand(Exponential(service_mean), num_customers)

    return arrival_times, service_times
end

generate_distributions (generic function with 1 method)

In [3]:
########################## MAIN SIMULATION #################################

# Main simulation function
function simulate_queue(arrival_rate::Float64, service_mean::Float64, num_customers::Int, num_servers::Int, max_server_capacity::Int, max_queue_length::Int, time_limit::Float64, verbose::Bool=false)
    queue_state = init_queue_state(num_servers, max_server_capacity, max_queue_length)
    arrival_times, service_times = generate_distributions(
    num_customers, 
    arrival_rate, 
    service_mean,
    )
    events, total_planned_time = generate_events(
    arrival_times, 
    service_times, 
    num_servers,
    )
    
    for event in events
        if !process_event!(event, queue_state, time_limit, verbose)
            break
        end
    end
    println("Total planned simulation time: ", total_planned_time)
end

simulate_queue (generic function with 2 methods)

In [4]:
########################### PARAMETERS #############################

# Parameters for generating distributions
arrival_rate = 50.0 / 60.0 # customers per minute (50 per hour converted)
CloakFloat_service_mean = 1.25 # service time in minutes
CloakSloth_service_mean = 0.8333 # service time in minutes
num_customers = 50

# Parameters for simulating the queue
CloakFloat_num_servers = 3
CloakSloth_num_servers = 2
max_server_capacity = 1
max_queue_length = 10000 # very large number
time_limit = 100.0 # very large number

100.0

In [5]:
########################### SIMULATION: CloakFloat #############################

simulate_queue(
arrival_rate, 
CloakFloat_service_mean, 
num_customers,
CloakFloat_num_servers, 
max_server_capacity, 
max_queue_length,
time_limit,
true
)

<br />########### Event: 1 ###########
Customer ID: 1
Current time: 0.5103456272665138
Customer 1 is arriving at server 1.
System state before event:
  Server 1: In Service = 0, Queue Length = 0
  Server 2: In Service = 0, Queue Length = 0
  Server 3: In Service = 0, Queue Length = 0
Total lost customers so far: 0
<br />Deviations:
System state after event:
Current time: 0.5103456272665138
  Server 1: In Service = 1, Queue Length = 0
  Server 2: In Service = 0, Queue Length = 0
  Server 3: In Service = 0, Queue Length = 0
Total lost customers: 0
Total waiting time in queue: 0.0
Total simulation time: 0.5103456272665138
Average queue length: 0.0
Sorted state times (State -> Time Spent):
  State 0: 0.5103456272665138

<br />########### Event: 3 ###########
Customer ID: 2
Current time: 0.9340097507304249
Customer 2 is arriving at server 2.
System state before event:
  Server 1: In Service = 1, Queue Length = 0
  Server 2: In Service = 0, Queue Length = 0
  Server 3: In Service = 0, Queue 

In [6]:
########################### SIMULATION: CloakSloth #############################

simulate_queue(
arrival_rate, 
CloakSloth_service_mean, 
num_customers,
CloakSloth_num_servers, 
max_server_capacity, 
max_queue_length,
time_limit,
true,
)

<br />########### Event: 1 ###########
Customer ID: 1
Current time: 1.1970031244303838
Customer 1 is arriving at server 1.
System state before event:
  Server 1: In Service = 0, Queue Length = 0
  Server 2: In Service = 0, Queue Length = 0
Total lost customers so far: 0
<br />Deviations:
System state after event:
Current time: 1.1970031244303838
  Server 1: In Service = 1, Queue Length = 0
  Server 2: In Service = 0, Queue Length = 0
Total lost customers: 0
Total waiting time in queue: 0.0
Total simulation time: 1.1970031244303838
Average queue length: 0.0
Sorted state times (State -> Time Spent):
  State 0: 1.1970031244303838

<br />########### Event: 2 ###########
Customer ID: 1
Current time: 1.2628813666293992
Customer 1 is leaving from server 1.
System state before event:
  Server 1: In Service = 1, Queue Length = 0
  Server 2: In Service = 0, Queue Length = 0
Total lost customers so far: 0
<br />Deviations:
System state after event:
Current time: 1.2628813666293992
  Server 1: In 

# Solving the tasks

## The statistics:

### 1. CloakFloat

Total waiting time in queue: 3.997855249056949

Total simulation time: 7.408373443192805

Average queue length: 0.5396400815526361

System state before event:

  Server 1: In Service = 1, Queue Length = 0

  Server 2: In Service = 1, Queue Length = 0
  
  Server 3: In Service = 1, Queue Length = 0

Steady state reached at time: 7.626831134748431

### 2. CloakSloth

Total waiting time in queue: 1.7133937228962726

Total simulation time: 10.320377180900767

Average queue length: 0.1660204557317087


System state before event:

  Server 1: In Service = 1, Queue Length = 0

  Server 2: In Service = 1, Queue Length = 0

Steady state reached at time: 10.974251600487444

## Oppgave a:

Steady state condition is set to reach a 0.10 deviation in avg queue length in the last 5 events happening in the system. I used hm5 as inspiration for setting up a steady state condition instead of calulcating with littles formulas.

The steady state conditon is reached in 10.32 for the CloakSloth and 7.49 for the CloakFloat.

When we multiply the simulation time which is the total time in the system, with the arrival rate we get the total number of customers within our system.

L_ClothSloth = λ⋅W = 0.8333⋅10.32 ≈ 8.59 customers

L_CloakFloat = λ⋅W = 0.8333⋅7.49 ≈ 6.24 customers

The CloakFloat has a average queue length on 0.54 customers and the average number of customers in the system is 6.24.

The average queue length is lower for the CloakSloth, which is 0.16 customers and the average number of customers is 7.49.

## Oppgave b:

For CloakFloat (3 employees):

Total operations per hour = 50 (arrival rate)

Total cost per hour = 50 * €23.50 = €1175

Total cost before reaching steady state = 6.24 * €23.50 = €146.64


For ClothesSloth (2 employees):

Total operations per hour = 50 (arrival rate)

Total cost per hour = 7.49 * €20.90 = €1045

Total cost before reaching steady state = 50 * €20.90 = €156.54

CloakSloth has the lowest waiting time in the system but it uses 44% percent longer time to reach a steady state compared with the cloackFloat system. This means that CloakFloat reaches a state where it maximizes the utilization rate before the CloakSloth system. The CloakFloat is less expensive too looking at the total cost when it has reached a steady state. Given that the total time the system runs is lower and that it reaches a steady state where the deviation of the last 5 events is less than 10 percent, i would recommend CloakFloat.